In [2]:
#模型构造

import torch
from torch import nn
from torch.nn import functional as F

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"  #设置选项允许重复加载动态链接库。


In [3]:
net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X = torch.rand(2,20)
net(X)

tensor([[-0.1223,  0.2924,  0.1159, -0.2223,  0.3256, -0.1173,  0.2562,  0.2867,
         -0.1382,  0.1224],
        [-0.1011,  0.0716, -0.0130, -0.1831,  0.2689, -0.1370,  0.2126,  0.2055,
         -0.0019,  0.0304]], grad_fn=<AddmmBackward0>)

In [9]:
#自定义块
class MLP(nn.Module):  #自定义MLP，是nnmodule的子类
    def __init__(self):      #在init函数中定义需要哪些类，和参数
        super().__init__()   #调用父类
        self.hidden = nn.Linear(20,256)  #全连接层：隐藏层
        self.out = nn.Linear(256,10)   #全连接层：输出层

    def forward(self,X):   #定义前向函数怎么运算 
        return self.out(F.relu(self.hidden(X)))#  X--hidden--relu--out


In [10]:
#使用上面定义的块

net = MLP()
net(X)

tensor([[-0.0784, -0.1840, -0.1328, -0.2348, -0.1102,  0.1692,  0.1330,  0.1058,
         -0.1576, -0.0452],
        [-0.1633, -0.1526, -0.1676, -0.1633, -0.0801,  0.1165,  0.1838,  0.0274,
         -0.2231,  0.0072]], grad_fn=<AddmmBackward0>)

In [11]:
#顺序块，，更仔细地看看Sequential类是如何工作的
# 构建我们自己的简化的MySequential


class MySequential(nn.Module):
    def __init__(self,*args):  #*args是收集参数，相当于吧若干个参数打包成一个来传入
        super().__init__()
        for block in args:
            self._modules[block] = block
             # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            #把各种层按顺序放在这个里面

    def forward(self,X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.1016,  0.0640,  0.3661, -0.0459,  0.0236,  0.2812, -0.1816, -0.0957,
         -0.1450,  0.0476],
        [-0.1098, -0.0439,  0.3525, -0.0350, -0.1401,  0.3229, -0.0149, -0.1691,
          0.0146,  0.0899]], grad_fn=<AddmmBackward0>)

In [12]:
#在正向传播函数中执行代码

class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20,20),requires_grad=True)
        self.linear = nn.Linear(20,20)
    def forward(self,X):
        X = self.linear(X)
        X = F.relu(torch.mm(X,self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)


tensor(0.1011, grad_fn=<SumBackward0>)

In [14]:
#混合搭配各种块的方法

class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),nn.ReLU(),
                                 nn.Linear(64,32),nn.ReLU())
        self.linear = nn.Linear(32,16)

    def forward(self,X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(),nn.Linear(16,20),FixedHiddenMLP())
chimera(X)

tensor(-0.1925, grad_fn=<SumBackward0>)

In [16]:
#参数管理

net = nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X = torch.rand(size = (2,4))
net(X)

tensor([[0.0952],
        [0.0633]], grad_fn=<AddmmBackward0>)

In [17]:
#参数访问
print(net[2].state_dict())   #net[2]拿到的是net（）第三个参数/输出层，然后state_dict是状态，取出W和B

OrderedDict([('weight', tensor([[ 0.1632,  0.1024, -0.1691,  0.1731,  0.2600,  0.2739,  0.1197, -0.2864]])), ('bias', tensor([0.1309]))])


In [18]:
print(type(net[2].bias))  #类型
print(net[2].bias)  #参数的状态
print(net[2].bias.data) #参数的值

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.1309], requires_grad=True)
tensor([0.1309])


In [19]:
net[2].weight.grad == None

True

In [20]:
#一次性访问所有参数
#*的作用是解包，将*后面的元组分解成一个一个的单元
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [21]:
#通过名字获取参数
net.state_dict()['2.bias'].data

tensor([0.1309])

In [22]:
#从嵌套块收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add_module(f'block {i}', block1())   #传入块的名字
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[-0.3435],
        [-0.3435]], grad_fn=<AddmmBackward0>)

In [23]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [24]:
rgnet[0][1][0].bias.data
#访问第一个主要的块中、第二个子块的第一层的偏置项

tensor([ 0.1459, -0.1774,  0.1376, -0.0099,  0.2416, -0.4510,  0.0322, -0.4758])

In [25]:
#内置参数初始化
def init_normal(m):   #normal正态分布，m是一个模租
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)   #对net里的所有参数调用init_normal函数 
net[0].weight.data[0], net[0].bias.data[0]

(tensor([ 0.0111, -0.0069, -0.0153, -0.0202]), tensor(0.))

In [26]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [27]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.3941, -0.2432,  0.3954, -0.6844])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [28]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -0.0000,  6.2045],
        [-0.0000, -8.2377, -7.3579, -9.7914]], grad_fn=<SliceBackward0>)

In [29]:
# 我们需要给共享层一个名称，以便可以引用它的参数
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])


tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [30]:
#自定义层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()   #super是调用父类函数

    def forward(self, X):
        return X - X.mean()

In [31]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [32]:
#将层作为组件合并到构建更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())
Y = net(torch.rand(4, 8))
Y.mean()

tensor(4.6566e-09, grad_fn=<MeanBackward0>)

In [33]:
#带参数的层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [34]:
linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.5549,  0.1034,  0.3348],
        [-0.4087,  1.6960, -0.3059],
        [ 1.4507,  0.8487, -0.9605],
        [-0.6297,  0.6909,  0.3398],
        [ 1.1423, -0.6707,  1.5335]], requires_grad=True)

In [35]:
linear(torch.rand(2, 5))

tensor([[2.0301, 0.0000, 0.9438],
        [1.8486, 0.0630, 0.9800]])

In [36]:
#使用自定义的层构建模型，就像使用内置的全连接层一样使用自定义层。
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))


tensor([[3.6955],
        [2.4422]])

In [38]:
#读写文件
# 可以直接调用load和save函数分别读写它们。 这两个函数都要求我们提供一个名称，save要求将要保存的变量作为输入。

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load("x-file")
x2

tensor([0, 1, 2, 3])

In [39]:
#存储一个张量列表，然后把他们都会内存

y = torch.zeros(4)
torch.save([x,y],'x-files')
x2,y2 = torch.load('x-files')

(x2,y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [40]:
#写入或者读取从字符串映射到张量的字典

mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [41]:
#加载和保存模型参数
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [42]:
#将模型的参数存储在一个叫做“mlp.params”的文件中
torch.save(net.state_dict(), 'mlp.params')

In [43]:
#为了恢复模型，我们实例化了原始多层感知机模型的一个备份。 
# 这里我们不需要随机初始化模型参数，而是直接读取文件中存储的参数。
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()   #eval()是进入测试模式

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [44]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])